In [3]:
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
print("xgboost", xgboost.__version__)

xgboost 1.7.5


In [4]:
model = XGBRegressor()

In [5]:
#use csv with already preprocessed features (scaling, differnecing etc needed)
df = pd.read_csv("Data_Prep.csv", index_col=0)
# Preprocessing for Prophet
my_df = df.reset_index() [["index", "Total_Heat"]]
my_df.columns = ["ds", "y"]
my_df.ds = pd.to_datetime(my_df.ds).dt.tz_convert(None)
df_scaler = MinMaxScaler()
my_df.y = pd.DataFrame(df_scaler.fit_transform(my_df[["y"]]), columns=["y"])


In [6]:
#use csv with already preprocessed features (scaling, differnecing etc needed)
df = pd.read_csv("Data_Prep.csv", index_col=0)

In [7]:
# Preprocessing for Prophet
my_df = df.reset_index() [["index", "Total_Heat"]]
my_df.columns = ["ds", "y"]
my_df.ds = pd.to_datetime(my_df.ds).dt.tz_convert(None)

In [8]:
df_scaler = MinMaxScaler()
my_df.y = pd.DataFrame(df_scaler.fit_transform(my_df[["y"]]), columns=["y"])


In [11]:
# Hyperparameter
train_days = 30
forecast_days = 2

In [12]:
# creating two seperate lists with a function

def create_X_y(data_df: pd.DataFrame, train_days:int, forecast_days:int):
    """"""
    X_list = []
    y_list = []

    for i in range(train_days*24, len(data_df)-forecast_days*24, 24):
        X_list.append(data_df.iloc[i-train_days*24:i])
        y_list.append(data_df[i:i+forecast_days*24])
    
    return X_list, y_list
    

In [13]:
X_list, y_list = create_X_y(my_df[["ds", "y"]], train_days, forecast_days)

In [14]:
X_list[0]

,ds,y
0,2008-01-01 00:00:00,0.362881
1,2008-01-01 01:00:00,0.371961
2,2008-01-01 02:00:00,0.397763
3,2008-01-01 03:00:00,0.449547
4,2008-01-01 04:00:00,0.594798
...,...,...
715,2008-01-30 19:00:00,0.463040
716,2008-01-30 20:00:00,0.417460
717,2008-01-30 21:00:00,0.325592
718,2008-01-30 22:00:00,0.263363


In [ ]:
for i in range(100):
    model.fit(trainX, trainy)

In [ ]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction
		yhat = xgboost_forecast(history, testX)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
		# summarize progress
		print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, 1], predictions